In [ ]:
import os
import yaml
from ultralytics import YOLO
import multiprocessing as mp
import pickle
import json

# Set multiprocessing start method to spawn for CUDA compatibility
mp.set_start_method('spawn', force=True)

# Configuration constants
CONFIG_YAML_FOLDER = 'dataset/training/config_yamls'
TRAINED_MODELS_FOLDER = 'runs'
K = 4  # Number of cross-validation folds

# Model configuration
base_model = 'yolo11'
sizes = ['n', 's', 'm', 'l'] 
models = {'_det': '', '_seg': '-seg', '_pose': '-pose'}
extensions = ['.pt', '.yaml']
tasks = {'_det': 'pieces', '_seg': 'board', '_pose': 'board'}

# Load previous results if available
try:
    with open('results.json', 'r') as f:
        results = json.load(f)
        print('Loaded previous results from results.json')
except Exception as e:
    results = {}
    print('No previous results found, starting fresh.', e)

# Initialize error tracking
try:
    with open('errors.txt', 'r') as f:
        errors = [line.strip() for line in f.readlines()]
        print('Loaded previous errors from errors.txt')
except Exception:
    errors = []
    print('No previous errors found, starting fresh.')

In [ ]:
from yolo_launcher import yolo_subprocess_val
from positions import *

def detection_metrics(base_model, model, size, ext):
    """
    Calculate detection metrics for piece detection models using cross-validation.
    
    Args:
        base_model: Base YOLO model name (e.g., 'yolo11')
        model: Model variant ('_det', '_seg', '_pose')
        size: Model size ('n', 's', 'm', 'l')
        ext: File extension ('.pt', '.yaml')
    
    Returns:
        Dictionary containing averaged metrics and variance across folds
    """
    metrics_list = []

    for k in range(K):
        # Construct model file path
        model_file_name = os.path.join(
            TRAINED_MODELS_FOLDER,
            'pieces',
            f'{base_model}{size}{models[model]}_k{k}_{ext[1:]}',
            'weights/best.pt'
        )
        
        # Get validation config file
        config_yaml_path = os.path.join(
            CONFIG_YAML_FOLDER,
            f'fold_{k}{model}.yaml'
        )
        if os.path.exists(config_yaml_path):
            val_path = config_yaml_path
        else:
            errors.append(f'Config YAML not found: {config_yaml_path}')
            continue
        # Load model and run validation to get metrics
        if os.path.exists(model_file_name):
            try:
                print(f'Loading model from {model_file_name} for fold {k}')

                proc = mp.Process(target=yolo_subprocess_val, args=(model_file_name, val_path))
                proc.start()
                proc.join()   

                with open('/tmp/yolo_results.pkl', 'rb') as f:
                    metrics = pickle.load(f)

                metrics_list.append(metrics)
            except Exception as e:
                errors.append(f'Error processing {model_file_name} for fold {k}: {e}')
        else:
            errors.append(f'Model file not found: {model_file_name}')
            continue

    # Calculate mean and variance for each metric
    map_values = [metrics.box.map for metrics in metrics_list]
    mp_values = [metrics.box.mp for metrics in metrics_list]
    mr_values = [metrics.box.mr for metrics in metrics_list]

    avg_metrics = {
        'mAP50-95': np.mean(map_values),
        'mAP50-95_var': np.var(map_values, ddof=1) if len(map_values) > 1 else 0,
        'mP': np.mean(mp_values),
        'mP_var': np.var(mp_values, ddof=1) if len(mp_values) > 1 else 0,
        'mR': np.mean(mr_values),
        'mR_var': np.var(mr_values, ddof=1) if len(mr_values) > 1 else 0,
        'best_fold': -1
    }
    
    best_map = 0
    for i in range(len(metrics_list)):
        if metrics_list[i].box.map > best_map:
            best_map = metrics_list[i].box.map
            avg_metrics['best_fold'] = i

    return avg_metrics

In [ ]:
# Functions to calculate metrics for board detection
import json
import cv2
import torch

from yolo_launcher import yolo_subprocess_batch

ORIGINAL_DATASET_LABEL_FOLDER = 'dataset/data/' # Using json labels for semplicity

def order_vertices(vertices):
    """Order vertices in clockwise order starting from top-left"""
    vertices = np.array(vertices).reshape(-1, 2)
    
    # Find centroid
    center = np.mean(vertices, axis=0)
    
    # Calculate angles from centroid
    angles = np.arctan2(vertices[:, 1] - center[1], vertices[:, 0] - center[0])
    
    # Sort by angle (clockwise)
    sorted_indices = np.argsort(angles)
    return vertices[sorted_indices]


def polygon_area(vertices):
    """Calculate area of polygon using shoelace formula"""
    vertices = np.array(vertices)
    n = len(vertices)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += vertices[i][0] * vertices[j][1]
        area -= vertices[j][0] * vertices[i][1]
    return abs(area) / 2.0

def clip_polygon_sutherland_hodgman(subject_polygon, clip_polygon):
    """Clip subject polygon by clip polygon using Sutherland-Hodgman algorithm"""
    def inside(p, cp1, cp2):
        return (cp2[0] - cp1[0]) * (p[1] - cp1[1]) > (cp2[1] - cp1[1]) * (p[0] - cp1[0])
    
    def compute_intersection(cp1, cp2, s, e):
        dc = [cp1[0] - cp2[0], cp1[1] - cp2[1]]
        dp = [s[0] - e[0], s[1] - e[1]]
        n1 = cp1[0] * cp2[1] - cp1[1] * cp2[0]
        n2 = s[0] * e[1] - s[1] * e[0]
        n3 = 1.0 / (dc[0] * dp[1] - dc[1] * dp[0])
        return [(n1 * dp[0] - n2 * dc[0]) * n3, (n1 * dp[1] - n2 * dc[1]) * n3]
    
    output_list = list(subject_polygon)
    cp1 = clip_polygon[-1]
    
    for cp2 in clip_polygon:
        input_list = output_list
        output_list = []
        if input_list:
            s = input_list[-1]
            for e in input_list:
                if inside(e, cp1, cp2):
                    if not inside(s, cp1, cp2):
                        output_list.append(compute_intersection(cp1, cp2, s, e))
                    output_list.append(e)
                elif inside(s, cp1, cp2):
                    output_list.append(compute_intersection(cp1, cp2, s, e))
                s = e
        cp1 = cp2
    
    return output_list

def calculate_quadrilateral_iou(quad1, quad2):
    """Calculate IoU between two quadrilaterals"""
    # Order vertices consistently
    quad1_ordered = order_vertices(quad1)
    quad2_ordered = order_vertices(quad2)
    
    # Find intersection using Sutherland-Hodgman clipping
    intersection = clip_polygon_sutherland_hodgman(quad1_ordered, quad2_ordered)
    
    if len(intersection) < 3:
        return 0.0
    
    # Calculate areas
    area1 = polygon_area(quad1_ordered)
    area2 = polygon_area(quad2_ordered)
    intersection_area = polygon_area(intersection)
    
    # Calculate IoU
    union_area = area1 + area2 - intersection_area
    if union_area == 0:
        return 0.0
    
    return intersection_area / union_area




def board_metrics(base_model, model, size, ext):
    avg_metrics = {'iou': 0, 'iou_transform': 0, 'best_fold': -1}
    best_metric = 0
    debug_avg_metrics = {}
    
    # Lists to store values from all folds for variance calculation
    all_fold_ious = []
    all_fold_ious_transform = []

    for k in range(K):
        model_file_name = os.path.join(
            TRAINED_MODELS_FOLDER,
            'board',
            f'{base_model}{size}{models[model]}_k{k}_{ext[1:]}',
            'weights/best.pt'
        )
        config_yaml_path = os.path.join(
            CONFIG_YAML_FOLDER,
            f'fold_{k}{model}.yaml'
        )
        if os.path.exists(config_yaml_path):
            with open(config_yaml_path, 'r') as file:
                config = yaml.safe_load(file)
                base_path = config['path']
                val_path = config['val']
        else:
            errors.append(f'Config YAML not found: {config_yaml_path}')
            return None, None
        images_path = os.path.join(base_path, val_path, 'images')

        # Get full path of images and labels listing the directory
        images = [os.path.join(images_path, f) for f in os.listdir(images_path) if f.endswith('.png')]

        # Load model once per fold
        #model_instance = YOLO(model_file_name)
        
        # Process images in batches to avoid memory issues
        batch_size = 32  # Adjust based on available GPU memory
        fold_ious = []
        fold_ious_transform = []
        
        for batch_start in range(0, len(images), batch_size):
            batch_end = min(batch_start + batch_size, len(images))
            batch_images = images[batch_start:batch_end]
            
            # Clear GPU cache before processing batch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            # run yolo_subprocess as a separate process and wait for it to finish
            proc = mp.Process(target=yolo_subprocess_batch, args=(model_file_name, batch_images))
            proc.start()
            proc.join()   

            with open('/tmp/yolo_results.pkl', 'rb') as f:
                model_results = pickle.load(f)

            for i, image in enumerate(batch_images):
                label = os.path.join(ORIGINAL_DATASET_LABEL_FOLDER, os.path.basename(image).replace('.png', '.json'))

                with open(label, 'r') as f:
                    correct_board_vert = json.load(f)['corners']

                if model == '_seg':                
                    if model_results[i].masks.xy is not None and len(model_results[i].masks.xy) > 0:
                        # Get the original mask contours from xy coordinates
                        mask_contours = model_results[i].masks.xy[0]
                        
                        # Convert to numpy array for OpenCV operations
                        contour_points = np.array(mask_contours, dtype=np.float32)
                        
                        # Approximate the contour to a quadrilateral using masks.xy
                        epsilon = 0.05 * cv2.arcLength(contour_points, True)
                        board_vert = cv2.approxPolyDP(contour_points, epsilon, True)
                        board_vert = board_vert.reshape(-1, 2)  # Flatten to 2D array
                elif model == '_pose':
                    board_vert = model_results[i].keypoints.xy[0].cpu().numpy()
                
                '''
                image_bgr = cv2.imread(image)
                image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
                approx_draw = np.array(board_vert, dtype=np.int32)

                # Draw approximated quadrilateral in green
                cv2.drawContours(image_rgb, [approx_draw], -1, (0, 255, 0), 3)
                
                # Save the image with both contours
                cv2.imwrite('/tmp/contours_comparison2.png', cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
                #print(f'Saved image with contours to /tmp/contours_comparison.png')
                '''

                print(f'Board vertices from model: {board_vert}')
                print(f'Correct board vertices: {correct_board_vert}')

                # Calculate standard IoU
                iou = calculate_quadrilateral_iou(board_vert, correct_board_vert)
                fold_ious.append(iou)
                print(f'IoU: {iou}')

                # Calculate IoU with transformation to board space
                try:
                    # Transform predicted vertices to board space using ground truth corners
                    transform_matrix = calc_transform(correct_board_vert)
                    
                    # Apply transformation to predicted vertices
                    board_vert_homogeneous = np.column_stack([board_vert, np.ones(len(board_vert))])
                    transformed_pred = np.dot(transform_matrix, board_vert_homogeneous.T).T
                    # Convert from homogeneous coordinates
                    transformed_pred = transformed_pred[:, :2] / transformed_pred[:, 2:]
                    
                    # Ground truth in board space is always a unit square
                    unit_square = np.array([[0, 0], [1, 0], [1, 1], [0, 1]])
                    
                    # Calculate IoU in transformed space
                    iou_transform = calculate_quadrilateral_iou(transformed_pred, unit_square)
                    fold_ious_transform.append(iou_transform)
                    print(f'IoU Transform: {iou_transform}')
                    
                except Exception as e:
                    print(f'Error calculating transform IoU: {e}')
                    fold_ious_transform.append(0.0)

            print(f'Done processing batch {batch_start // batch_size + 1}/{(len(images) + batch_size - 1) // batch_size} for fold {k}, {model}, {size}, {ext}')

        # Calculate average IoU for this fold
        if fold_ious:
            fold_avg_iou = sum(fold_ious) / len(images)
            fold_avg_iou_transform = sum(fold_ious_transform) / len(images)

            debug_avg_metrics[f'fold_{k}'] = {
                'iou': fold_avg_iou,
                'iou_transform': fold_avg_iou_transform
            }
            
            # Store fold averages for variance calculation
            all_fold_ious.append(fold_avg_iou)
            all_fold_ious_transform.append(fold_avg_iou_transform)
            
            avg_metrics['iou'] += fold_avg_iou
            avg_metrics['iou_transform'] += fold_avg_iou_transform
            
            if fold_avg_iou_transform > best_metric:
                best_metric = fold_avg_iou_transform
                avg_metrics['best_fold'] = k
    
    # Calculate final averages and variances
    avg_metrics['iou'] /= K
    avg_metrics['iou_transform'] /= K
    avg_metrics['iou_var'] = np.var(all_fold_ious, ddof=1) if len(all_fold_ious) > 1 else 0
    avg_metrics['iou_transform_var'] = np.var(all_fold_ious_transform, ddof=1) if len(all_fold_ious_transform) > 1 else 0
    
    return avg_metrics

In [ ]:
if __name__ == "__main__":
    for ext in extensions:
        for model, model_ext in models.items():
            task = tasks[model]
            print(f'Processing task: {task}, model: {model}, extension: {ext}')
            for size in sizes:
                if task not in results:
                    results[task] = {}

                if f'{size}{model}{ext}' in results[task]:
                    print(f'Skipping {size}{model}{ext} as it already exists in results')
                    continue

                if model == '_det':
                    avg_metrics = detection_metrics(base_model, model, size, ext)
                else:
                    avg_metrics = board_metrics(base_model, model, size, ext)

                results[task][f'{size}{model}{ext}'] = avg_metrics

                torch.cuda.empty_cache()
                torch.cuda.ipc_collect()

                # Save results to json/pickle
                with open('results.json', 'w') as f:
                    json.dump(results, f, indent=4)
                with open('errors.txt', 'w') as f:
                    for error in errors:
                        f.write(f'{error}\n')

    print('Errors:', errors)